05 Jun 2017

Convolution Test Features -- Test.

Following along: [[here](https://gist.github.com/idanori/3dadb3c1d66223170d2df4cd802da7cf)]

In [ ]:
import theano

In [ ]:
%matplotlib inline
from __future__ import print_function, division

import utils
from utils import *
from IPython.display import FileLink
from shutil import copyfile, move
import pandas as pd
import matplotlib.image as mpimg

In [ ]:
path = "data/statefarm/"

In [ ]:
batch_size = 16

### Save Array Function

In [ ]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]
def load_carray(fname): return bcolz.open(fname)

In [ ]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
def hotone(x):
    ret = []
    for i in range(len(x)):
        for j in range(len(x[i])):
            if x[i][j] == 1:
                ret += [j]
    return ret
# concat all arrays from batch dir
def concat_batches(batches):
    return np.concatenate([batches.next() for i in range(int(batches.n/batches.batch_size+1))])

### Add Conv Layer

In [ ]:
import vgg16
vggModel = Vgg16()
vggModel.model.summary()

In [ ]:
# get index of last conv layer
lastConvIndex = -1
for i in range(len(vggModel.model.layers)):
    if (type(vggModel.model.layers[i]) == Convolution2D):
        lastConvIndex = i

print ('last conv layer index ' + str(i))

In [ ]:
convLayers = vggModel.model.layers[:lastConvIndex+1]
vggConvLayersModel = Sequential(convLayers)
vggConvLayersModel.summary()

In [ ]:
#generate an augmented train batch
imgGen = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                                  shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

augmentTrainBatch = imgGen.flow_from_directory(directory=path+'train', batch_size=64, target_size=(224,224), 
                                               shuffle=False, class_mode='categorical')

In [ ]:
train_batches = augmentTrainBatch

# generate a new validation batch
val_batches = get_batches(path + 'valid', batch_size=batch_size, shuffle=False, 
                         target_size=(224,224))

In [ ]:
test_batch = get_batches(path + 'test', batch_size=batch_size, shuffle=False,
                        target_size=(224,224), class_mode='binary')

In [ ]:
val_batches.reset()
train_label_arr = onehot(train_batches.classes)
train_file_names_arr = train_batches.filenames
valid_label_arr = onehot(val_batches.classes)
valid_file_names_arr = val_batches.filenames
test_file_names_arr = test_batch.filenames

#### predict batches on pretrained model (based on conv layer of VGG16)

In [ ]:
predictTrainVggConv = vggConvLayersModel.predict_generator(test_batch, test_batch.n)